In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import nltk

### NLTK expects sentences, so let's recover sentences from the .testb file

In [92]:
# Format of the .train .test files:
#
# SOCCER NN I-NP O
# ...
#
# (word) (Part of speech) (IOB2 tag) (end sequence character)

In [56]:
with open('eng.testb') as fl:
    sents = fl.read().split('\n\n')
print(len(sents))

3685


In [57]:
sents[0], sents[-1] # header, footer

('-DOCSTART- -X- -X- O', '')

In [58]:
sents = sents[1:-1]
sents = [meta.split('\n') for meta in sents]

### WARN: Don't join then re-tokenize the words b/c NLTK may produce a different tokenization than what is provided

In [66]:
def recover_sent(meta):
    # The first words compose a sentence (or a phrase)
    
    # TODO:

    return words

print(sents[0])
recover_sent(sents[0])

['SOCCER NN I-NP O', '- : O O', 'JAPAN NNP I-NP I-LOC', 'GET VB I-VP O', 'LUCKY NNP I-NP O', 'WIN NNP I-NP O', ', , O O', 'CHINA NNP I-NP I-PER', 'IN IN I-PP O', 'SURPRISE DT I-NP O', 'DEFEAT NN I-NP O', '. . O O']


['SOCCER',
 '-',
 'JAPAN',
 'GET',
 'LUCKY',
 'WIN',
 ',',
 'CHINA',
 'IN',
 'SURPRISE',
 'DEFEAT',
 '.']

In [60]:
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk

In [83]:
test_me = recover_sent(sents[0])

# HINT: use pos_tag, ne_chunk, and tree2conlltags from NLTK
def sent2IOB(tkns):
    # Uses conll tagger to get IOBs
    
    # TODO:
    
    return iob_tagged
    
print(sent2IOB(test_me))

[('SOCCER', 'NNP', 'B-GPE'), ('-', ':', 'O'), ('JAPAN', 'NNP', 'B-ORGANIZATION'), ('GET', 'NNP', 'O'), ('LUCKY', 'NNP', 'O'), ('WIN', 'NNP', 'O'), (',', ',', 'O'), ('CHINA', 'NNP', 'B-GPE'), ('IN', 'NNP', 'O'), ('SURPRISE', 'NNP', 'O'), ('DEFEAT', 'NNP', 'O'), ('.', '.', 'O')]


### The eng.train/test files use IOB1(?) such that only 4 tags (LOC, PER, ORG, MISC) + 1 null char (O) are used, but NLTK may return a few more ent types. 

<br>

### If that's the case, think of a mapping to the 4 types so that we can run the test.

In [88]:
toIOB2 = {
    # TODO: mappings by dictionary (FROM: TO)
}

### Code below dumps your results into a .output file. Feel free to edit or replace.

### (A reminder that conlleval.py expects each row to be `<word> <actual> <predicted>` with emtpy lines after each sentence)

#### i.e.
```
SOCCER O B-LOC
- O O
JAPAN I-LOC B-ORG
GET O O
LUCKY O O
WIN O O
, O O
CHINA I-PER B-LOC
IN O O
SURPRISE O O
DEFEAT O O
. O O
```

In [93]:
import os, sys

with open('eng_iob2.output', 'w') as fl:
    for si, meta in enumerate(sents):
        sent = recover_sent(meta)
        iobs = sent2IOB(sent)
        
        assert len(iobs) is len(meta)
        
        for iob, inp in zip(iobs, meta):
            # APPLY THE MAPPING
            mapped_iob = iob[2]
            if '-' in mapped_iob:
                ichar, etype = mapped_iob.split('-')
                mapped_iob = '%s-%s' % (ichar, toIOB2[etype])
            fl.write('%s %s %s\n' % (iob[0], inp.split()[3], mapped_iob))
        fl.write('\n')
#         break
        
        sys.stdout.write('[%d/%d]  \r' % (si, len(sents)))
    sys.stdout.flush()

In [91]:
!python conlleval.py eng_iob2.output

processed 46665 tokens with 5648 phrases; found: 5384 phrases; correct: 2630.
accuracy:  83.59%; precision:  48.85%; recall:  46.57%; FB1:  47.68
              LOC: precision:  51.60%; recall:  70.38%; FB1:  59.55  2275
             MISC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              ORG: precision:  32.09%; recall:  25.29%; FB1:  28.28  1309
              PER: precision:  57.56%; recall:  64.07%; FB1:  60.64  1800
